# Vector Borne disease
[Kaggle Competition - Playground Series S3E13](https://www.kaggle.com/competitions/playground-series-s3e13/)


    - 'id', 'sudden_fever', 'headache', 'mouth_bleed', 'nose_bleed',
       'muscle_pain', 'joint_pain', 'vomiting', 'rash', 'diarrhea',
       'hypotension', 'pleural_effusion', 'ascites', 'gastro_bleeding',
       'swelling', 'nausea', 'chills', 'myalgia', 'digestion_trouble',
       'fatigue', 'skin_lesions', 'stomach_pain', 'orbital_pain', 'neck_pain',
       'weakness', 'back_pain', 'weight_loss', 'gum_bleed', 'jaundice', 'coma',
       'diziness', 'inflammation', 'red_eyes', 'loss_of_appetite',
       'urination_loss', 'slow_heart_rate', 'abdominal_pain',
       'light_sensitivity', 'yellow_skin', 'yellow_eyes', 'facial_distortion',
       'microcephaly', 'rigor', 'bitter_tongue', 'convulsion', 'anemia',
       'cocacola_urine', 'hypoglycemia', 'prostraction', 'hyperpyrexia',
       'stiff_neck', 'irritability', 'confusion', 'tremor', 'paralysis',
       'lymph_swells', 'breathing_restriction', 'toe_inflammation',
       'finger_inflammation', 'lips_irritation', 'itchiness', 'ulcers',
       'toenail_loss', 'speech_problem', 'bullseye_rash', 'prognosis'

Prognosis: 11 categories
'Lyme_disease', 'Tungiasis', 'Zika', 'Rift_Valley_fever', 'West_Nile_fever', 'Malaria', 'Chikungunya', 'Plague', 'Dengue', 'Yellow_Fever', 'Japanese_encephalitis'

In [37]:

import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna

def convert(dataframe):
    dataframe.set_index("id", inplace=True)
    bool_columns = {col: 'bool' for col in dataframe.columns if col != 'prognosis'}
    dataframe = dataframe.astype(bool_columns)
    return dataframe

data_train = pd.read_csv("/home/rainer/Downloads/ML_datasets/vector_borne_disease_train.csv")
data_train = convert(data_train)
category_mapping = dict(enumerate(data_train["prognosis"].astype("category").cat.categories))
data_train["prognosis"]=data_train["prognosis"].astype("category").cat.codes


In [38]:
category_mapping

{0: 'Chikungunya',
 1: 'Dengue',
 2: 'Japanese_encephalitis',
 3: 'Lyme_disease',
 4: 'Malaria',
 5: 'Plague',
 6: 'Rift_Valley_fever',
 7: 'Tungiasis',
 8: 'West_Nile_fever',
 9: 'Yellow_Fever',
 10: 'Zika'}

In [39]:
X, Y = data_train.drop('prognosis', axis=1), data_train[['prognosis']]
dtrain = xgb.DMatrix(X, Y, enable_categorical=True)

In [40]:
def objective(trial):
    param = {
        'tree_method':'gpu_hist',
        "objective": "multi:softprob",
        "num_class": 11,
        'eta': trial.suggest_float('eta', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),

    }

    boostround=200
    results = xgb.cv(
        params=param,
        dtrain=dtrain,
        num_boost_round=boostround,
        nfold=5,
        early_stopping_rounds=50,
        verbose_eval=100, # print eval every xth round
        metrics=["mlogloss", "merror"],
    )

    return results.iloc[-1]["test-mlogloss-mean"]

In [41]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-05-01 09:08:01,208] A new study created in memory with name: no-name-41b03375-0393-49c9-8ac0-622953340923


[0]	train-mlogloss:2.11341+0.01130	train-merror:0.65063+0.00612	test-mlogloss:2.25450+0.03172	test-merror:0.77232+0.02626
[77]	train-mlogloss:0.13980+0.00380	train-merror:0.00530+0.00250	test-mlogloss:2.49276+0.17452	test-merror:0.70440+0.01484


[I 2023-05-01 09:08:08,583] Trial 0 finished with value: 2.2688244084510356 and parameters: {'eta': 0.4389590684994202, 'max_depth': 10, 'min_child_weight': 0.15748735169148229, 'subsample': 0.2685655505176055, 'colsample_bynode': 0.17969706900520588}. Best is trial 0 with value: 2.2688244084510356.


[0]	train-mlogloss:2.17246+0.01487	train-merror:0.60537+0.01318	test-mlogloss:2.28576+0.01076	test-merror:0.74114+0.03262
[92]	train-mlogloss:0.12770+0.00353	train-merror:0.00247+0.00180	test-mlogloss:2.39079+0.11215	test-merror:0.69449+0.02735


[I 2023-05-01 09:08:14,665] Trial 1 finished with value: 2.1526004058203307 and parameters: {'eta': 0.2603878934581738, 'max_depth': 4, 'min_child_weight': 0.6981742627785527, 'subsample': 0.4938587798348948, 'colsample_bynode': 0.9452254687142239}. Best is trial 1 with value: 2.1526004058203307.


...

[0]	train-mlogloss:2.30089+0.00469	train-merror:0.39675+0.00563	test-mlogloss:2.36396+0.00524	test-merror:0.75105+0.02993
[100]	train-mlogloss:0.19819+0.00405	train-merror:0.00742+0.00206	test-mlogloss:1.95721+0.05700	test-merror:0.68316+0.01391
[176]	train-mlogloss:0.07940+0.00244	train-merror:0.00212+0.00173	test-mlogloss:2.02712+0.06974	test-merror:0.68458+0.01716


[I 2023-05-01 09:51:51,881] Trial 198 finished with value: 1.981748509142485 and parameters: {'eta': 0.06971762755633565, 'max_depth': 8, 'min_child_weight': 0.025339995930820616, 'subsample': 0.7470500370748757, 'colsample_bynode': 0.19498795637674182}. Best is trial 172 with value: 1.9117700509383162.


[0]	train-mlogloss:2.31273+0.00272	train-merror:0.55409+0.01276	test-mlogloss:2.36141+0.00284	test-merror:0.77941+0.02570
[71]	train-mlogloss:0.48492+0.00756	train-merror:0.03430+0.00616	test-mlogloss:1.93112+0.07665	test-merror:0.68314+0.01568


[I 2023-05-01 09:51:58,617] Trial 199 finished with value: 2.0065433676370126 and parameters: {'eta': 0.08713203576758873, 'max_depth': 6, 'min_child_weight': 0.08028936116929673, 'subsample': 0.5807596724531539, 'colsample_bynode': 0.16063719075245092}. Best is trial 172 with value: 1.9117700509383162.


Number of finished trials: 200
Best trial:
  Value: 1.9117700509383162
  Params: 
    eta: 0.059011359743889734
    max_depth: 7
    min_child_weight: 0.017084361725073466
    subsample: 0.6521245117384575
    colsample_bynode: 0.10149306931302936


In [42]:
bestParams = study.best_trial.params
bestParams["tree_method"]= "gpu_hist"
bestParams["objective"]= "multi:softprob"
bestParams["num_class"]= 11
model = xgb.train(
    params=bestParams,
    dtrain=dtrain,
    num_boost_round=200
)

disease_test = pd.read_csv("/home/rainer/Downloads/ML_datasets/vector_borne_disease_test.csv")
disease_test = convert(disease_test)
disease_finalPred_matrix = xgb.DMatrix(disease_test, enable_categorical=False)
predictions = model.predict(disease_finalPred_matrix)


In [43]:
import numpy as np

def get_top_3_indices(row):
    return list(np.argsort(row.values)[-3:][::-1])

predictionsDf = pd.DataFrame(predictions)
predictionsDf['top_3_columns'] = predictionsDf.apply(get_top_3_indices, axis=1)
predictionsDf['top_3_categories'] = predictionsDf['top_3_columns'].apply(lambda x: [category_mapping[i] for i in x])
predictionsDf['prognosis'] = predictionsDf['top_3_categories'].apply(lambda x: ' '.join(x))
predictionsDf["id"] = disease_test.index


In [44]:
predictionsDf

,0,1,2,3,4,5,6,7,8,9,10,top_3_columns,top_3_categories,prognosis,id
0,0.055682,0.089040,0.109213,0.003736,0.004562,0.021320,0.480762,0.108483,0.028118,0.024505,0.074578,"[6, 2, 7]","[Rift_Valley_fever, Japanese_encephalitis, Tun...",Rift_Valley_fever Japanese_encephalitis Tungiasis,707
1,0.074087,0.525106,0.037462,0.005252,0.076476,0.130347,0.058015,0.013680,0.022498,0.043531,0.013546,"[1, 5, 4]","[Dengue, Plague, Malaria]",Dengue Plague Malaria,708
2,0.020564,0.024861,0.228787,0.024356,0.055232,0.025924,0.055285,0.040680,0.443302,0.057706,0.023303,"[8, 2, 9]","[West_Nile_fever, Japanese_encephalitis, Yello...",West_Nile_fever Japanese_encephalitis Yellow_F...,709
3,0.023508,0.075883,0.389935,0.009759,0.034428,0.018424,0.239104,0.079398,0.034732,0.051875,0.042954,"[2, 6, 7]","[Japanese_encephalitis, Rift_Valley_fever, Tun...",Japanese_encephalitis Rift_Valley_fever Tungiasis,710
4,0.011783,0.050656,0.128091,0.066302,0.129838,0.077843,0.054207,0.040666,0.276008,0.064696,0.099913,"[8, 4, 2]","[West_Nile_fever, Malaria, Japanese_encephalitis]",West_Nile_fever Malaria Japanese_encephalitis,711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.014121,0.004364,0.049991,0.009637,0.047538,0.075532,0.018413,0.083320,0.177662,0.277365,0.242058,"[9, 10, 8]","[Yellow_Fever, Zika, West_Nile_fever]",Yellow_Fever Zika West_Nile_fever,1005
299,0.002188,0.002763,0.006520,0.501704,0.405949,0.033915,0.006001,0.002371,0.026548,0.004508,0.007534,"[3, 4, 5]","[Lyme_disease, Malaria, Plague]",Lyme_disease Malaria Plague,1006
300,0.004314,0.014439,0.040066,0.440451,0.074599,0.042661,0.016485,0.018135,0.312981,0.022185,0.013684,"[3, 8, 4]","[Lyme_disease, West_Nile_fever, Malaria]",Lyme_disease West_Nile_fever Malaria,1007
301,0.012607,0.141119,0.093178,0.052928,0.020444,0.038383,0.211784,0.043949,0.153668,0.031741,0.200198,"[6, 10, 8]","[Rift_Valley_fever, Zika, West_Nile_fever]",Rift_Valley_fever Zika West_Nile_fever,1008


In [45]:
predictionsDf.to_csv('/home/rainer/Downloads/ML_datasets/vector_borne_disease_test_prediction.csv', columns=['id','prognosis'], index=False)